In [6]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [8]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/refs/heads/main/tweet_emotions.csv").drop(columns=["tweet_id"])
df.head()


,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


# Data Preprocessing

In [9]:
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove the stop word from text"""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return text

def removing_number(text):
    """Remove number from text"""
    text = "".join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case"""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Removing punctuations from the texts"""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace(" ","")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Removing URLs from the text"""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r"", text)


def normalize_text(df):
    """Normalize the text Data"""
    try:
        df['content']=df['content'].apply(lower_case)
        df['content']=df['content'].apply(remove_stop_words)
        df['content']=df['content'].apply(removing_number)
        df['content']=df['content'].apply(removing_punctuations)
        df['content']=df['content'].apply(removing_urls)
        df['content']=df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during normalization: {e}")
        raise             

<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\s'
C:\Users\user\AppData\Local\Temp\ipykernel_8224\1521413930.py:29: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [10]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylueknowlisteninbadhabitearlierstartedfr...
1,sadness,layinnbedheadacheughhhhwaitincall
2,sadness,funeralceremonygloomyfriday
3,enthusiasm,wantshangfriendssoon
4,neutral,dannycastillowanttradesomeonehoustonticketsone...


In [11]:
df["sentiment"].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [13]:
x = df["sentiment"].isin(["happiness", "sadness"])
df = df[x]

In [14]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_8224\1089524538.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layinnbedheadacheughhhhwaitincall
2,0,funeralceremonygloomyfriday
6,0,sleepimnotthinkingoldfriendwantmarriednowdamna...
8,0,charviraycharlenelovemiss
9,0,kelcouchsorryleastfriday


In [15]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
import dagshub



mlflow.set_tracking_uri("https://dagshub.com/varun19424/mini-mlops-project.mlflow")
dagshub.init(repo_owner='varun19424', repo_name='mini-mlops-project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\Varun\MyLearning\MLOps\MiniProject\mlops-mini-project\env\Lib\site-packages\rich\live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=73b5e0e1-88b7-4eb3-92d5-375980fcd5b5&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=effbf583f9db8fc4062d560985e11450324578995ca8df89d6384a36a3287b53




Accessing as varun19424

Initialized MLflow to track repo "varun19424/mini-mlops-project"

Repository varun19424/mini-mlops-project initialized!

2025/07/10 22:59:10 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/15377add6bd4484ca6de7d1bf69f06b0', creation_time=1752168551783, experiment_id='0', last_update_time=1752168551783, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
with mlflow.start_run():
    
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
   # Save and log the model manually
    import joblib
    joblib.dump(model, "model.pkl")
    mlflow.log_artifact("model.pkl")

    # Save and log the notebook
    import os
    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

Accuracy: 0.4939759036144578
Precision: 0.4915254237288136
Recall: 1.0
F1 Score: 0.6590909090909091
🏃 View run placid-bird-476 at: https://dagshub.com/varun19424/mini-mlops-project.mlflow/#/experiments/0/runs/ab1eed77b6114e37835b47f2299887b6
🧪 View experiment at: https://dagshub.com/varun19424/mini-mlops-project.mlflow/#/experiments/0


In [22]:
import os
os.remove("model.pkl")
